# Ontology creation for TCE/RN

In [14]:
import sys, os

from owlready2 import *

In [4]:
onto_path.append(os.path.expanduser("~/ontologias"))

In [16]:
world = World()
dolce = world.get_ontology("./DOLCE-Lite.owl").load()

In [19]:
dolce.SocialObject

In [12]:
tce = world.get_ontology("http://tce.rn.gov.br/tce.owl")
with tce:
    tce.imported_ontologies.append(dolce)

In [13]:
print("DOLCE carregado:", dolce.base_iri)
print("Classes DOLCE:", len(list(dolce.classes())))
print("Importações em tce:", [o.base_iri for o in tce.imported_ontologies])

# (opcional) salvar um snapshot para conferir no Protégé
world.save(file = "tce_imports.owl", format = "rdfxml")
print("Arquivo salvo:", os.path.abspath("tce_imports.owl"))

DOLCE carregado: http://www.loa-cnr.it/ontologies/DOLCE-Lite.owl#
Classes DOLCE: 37
Importações em tce: ['http://www.loa-cnr.it/ontologies/DOLCE-Lite.owl#']
Arquivo salvo: c:\Users\05911205424\Documents\Dev\decicontas.br\tce_imports.owl


In [17]:
with tce:
    # 1️⃣ Classe Processo
    class Processo(dolce.SocialObject):
        pass
    
    # 2️⃣ Classe PessoaResponsavel (agente)
    class PessoaResponsavel(dolce.AgentivePhysicalObject):
        pass

    # 3️⃣ Propriedades (object & data)
    class temResponsavel(ObjectProperty):
        domain = [Processo]
        range = [PessoaResponsavel]
    
    class criadoEm(DataProperty):
        domain = [Processo]
        range = [str]

# 4️⃣ Criar instâncias de teste
proc = tce.Processo("proc_1234")
proc.criadoEm = ["2023-10-05"]

resp = tce.PessoaResponsavel("joao_silva")
proc.temResponsavel = [resp]

# 5️⃣ Salvar ontologia resultante
world.save(file = "tce_processo.owl", format = "rdfxml")
print("Ontologia salva em:", os.path.abspath("tce_processo.owl"))

# 6️⃣ Conferir
print(f"\nProcesso: {proc.name}")
print(f"Responsável: {proc.temResponsavel[0].name}")
print(f"Data de criação: {proc.criadoEm[0]}")


TypeError: NoneType takes no arguments

In [20]:
from owlready2 import *

# assumindo que 'dolce' já está carregado
hits = [c for c in dolce.classes() if "social" in c.name.lower()]
for c in hits:
    print(c.name, "→", c.iri)
